In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import random
import re
import os
from PIL import Image
import io

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

def cargar_modelo_imagenes():
    modelo_base = MobileNetV2(weights='imagenet', include_top=True)
    return modelo_base

def preprocesar_imagen(ruta_imagen=None, imagen_pil=None):
    if ruta_imagen:
        img = image.load_img(ruta_imagen, target_size=(224, 224))
    elif imagen_pil:
        img = imagen_pil.resize((224, 224))
    else:
        raise ValueError("Debes proporcionar una ruta o una imagen PIL.")
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

lemmatizer = WordNetLemmatizer()
modelo_imagenes = cargar_modelo_imagenes()
print("Modelos base cargados correctamente.")

def analizar_imagen(ruta_imagen=None, imagen_pil=None):
    img_array = preprocesar_imagen(ruta_imagen, imagen_pil)
    if img_array is None:
        return "No se pudo procesar la imagen."
    preds = modelo_imagenes.predict(img_array)
    resultados = decode_predictions(preds, top=3)[0]
    descripcion = "En esta imagen puedo ver: "
    etiquetas = []
    for (_, nombre, _) in resultados:
        etiquetas.append(nombre.replace('_', ' '))
    descripcion += ", ".join(etiquetas) + "."
    contexto = interpretar_contexto_imagen(etiquetas)
    if contexto:
        descripcion += f" {contexto}"
    return descripcion

def interpretar_contexto_imagen(etiquetas):
    contexto = ""
    etiquetas = [e.lower() for e in etiquetas]
    if any(obj in etiquetas for obj in ['dog', 'cat', 'puppy', 'kitten']):
        contexto = "Parece una escena tranquila y amigable con animales."
    elif any(obj in etiquetas for obj in ['person', 'man', 'woman']):
        contexto = "Podría haber interacción humana en la imagen."
    elif any(obj in etiquetas for obj in ['forest', 'mountain', 'ocean', 'beach']):
        contexto = "La imagen muestra un entorno natural o relajante."
    elif any(obj in etiquetas for obj in ['car', 'bus', 'airplane']):
        contexto = "Parece una escena de transporte o movimiento."
    elif any(obj in etiquetas for obj in ['weapon', 'knife', 'gun']):
        contexto = "Advertencia: la imagen podría contener contenido sensible."
    return contexto

patrones_conversacion = {
    "saludos": [r"hola", r"buenos días", r"buenas tardes", r"hey", r"saludos"],
    "preguntas_imagen": [r"qué ves en (la|esta) imagen", r"describe (la|esta) imagen", r"qué hay en (la|esta) imagen", r"qué puedes ver", r"analiza (la|esta) imagen"],
    "consulta_psicologica": [r"me siento (triste|feliz|ansioso|preocupado|estresado)", r"tengo problemas con", r"no puedo (dormir|concentrarme|estudiar)", r"necesito ayuda con mis emociones", r"cómo puedo manejar (el estrés|la ansiedad|la depresión)"],
    "consulta_academica": [r"no entiendo (este tema|esta materia)", r"cómo puedo estudiar mejor", r"tengo dificultades con", r"necesito ayuda con mis estudios", r"cómo mejorar (mi concentración|mi memoria|mis notas)"],
    "despedida": [r"adiós", r"hasta luego", r"nos vemos", r"chao", r"bye"]
}

respuestas = {
    "saludos": ["\u00a1Hola! ¿En qué puedo ayudarte hoy?", "\u00a1Saludos! Soy un asistente virtual. Puedo analizar imágenes y conversar contigo.", "\u00a1Buen día! Estoy aquí para ayudarte con tus consultas e imágenes."],
    "preguntas_imagen_sin_contexto": ["Para analizar una imagen, necesito que me la proporciones primero.", "No tengo ninguna imagen para analizar. ¿Podrías compartir una?", "Necesito ver una imagen antes de poder describirla."],
    "consulta_psicologica": ["Entiendo cómo te sientes. ¿Podrías contarme más sobre eso?", "Es normal tener esos sentimientos. ¿Qué crees que los está causando?", "Gracias por compartir eso conmigo. Te escucho. ¿Desde cuándo te sientes así?"],
    "consulta_academica": ["Aprender puede ser desafiante. ¿Con qué tema específico estás teniendo dificultades?", "Cada persona tiene su propio estilo de aprendizaje. ¿Has identificado qué métodos funcionan mejor para ti?", "El éxito académico requiere estrategia. ¿Has probado crear un horario de estudio?"],
    "despedida": ["\u00a1Hasta pronto! Fue un placer ayudarte.", "\u00a1Adiós! Si necesitas más ayuda, aquí estaré.", "\u00a1Que tengas un excelente día! Regresa cuando necesites apoyo."],
    "default": ["Interesante. Cuéntame más.", "No estoy seguro de entender. ¿Podrías explicarlo de otra manera?", "Estoy aquí para ayudarte. ¿Puedes ser más específico?"]
}

def procesar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúñü0-9\s]', '', texto)
    tokens = word_tokenize(texto)
    tokens_lematizados = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens_lematizados

def identificar_intencion(texto):
    texto = texto.lower()
    for categoria, patrones in patrones_conversacion.items():
        for patron in patrones:
            if re.search(patron, texto):
                return categoria
    return "default"

class AsistenteVirtual:
    def __init__(self):
        self.modelo_imagenes = modelo_imagenes
        self.imagen_actual = None
        self.analisis_imagen_actual = None
        self.contexto_conversacion = []

    def procesar_imagen(self, ruta_imagen=None, imagen_pil=None):
        try:
            self.imagen_actual = imagen_pil if imagen_pil else Image.open(ruta_imagen)
            self.analisis_imagen_actual = analizar_imagen(ruta_imagen, imagen_pil)
            return self.analisis_imagen_actual
        except Exception as e:
            return f"Ocurrió un error al procesar la imagen: {str(e)}"

    def responder(self, texto_usuario):
        categoria = identificar_intencion(texto_usuario)
        if categoria == "preguntas_imagen":
            if self.analisis_imagen_actual:
                respuesta = self.analisis_imagen_actual
            else:
                respuesta = random.choice(respuestas["preguntas_imagen_sin_contexto"])
        elif categoria in respuestas:
            respuesta = random.choice(respuestas[categoria])
        else:
            respuesta = random.choice(respuestas["default"])
        self.contexto_conversacion.append({"usuario": texto_usuario, "respuesta": respuesta})
        return respuesta

from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

def generador_ejercicios(tema, dificultad):
    if tema.lower() == "matemáticas":
        if dificultad == "básico":
            return ["¿Cuánto es 3 + 4?", "Resuelve: 8 - 2"]
        elif dificultad == "intermedio":
            return ["Resuelve: 12 × 5", "Calcula: 72 ÷ 8"]
        elif dificultad == "avanzado":
            return ["Calcula la derivada de x^2", "Integra: ∫ x dx"]
    elif tema.lower() == "vocabulario":
        return ["Escribe una oración con la palabra 'amistad'.", "Define: 'empatía'"]
    return ["Lo siento, no tengo ejercicios para ese tema."]

def analisis_sentimiento(texto):
    sia = SentimentIntensityAnalyzer()
    score = sia.polarity_scores(texto)['compound']
    if score >= 0.5:
        return "positivo"
    elif score <= -0.5:
        return "negativo"
    else:
        return "neutral"

class AsistenteVirtualMejorado(AsistenteVirtual):
    def __init__(self):
        super().__init__()

    def analizar_sentimiento_usuario(self, texto):
        return analisis_sentimiento(texto)

    def recomendar_recursos(self, tema, sentimiento=None):
        if sentimiento == "negativo":
            return f"Te recomiendo un descanso y luego repasar lo básico de {tema}. Aquí tienes una guía práctica."
        elif sentimiento == "positivo":
            return f"¡Genial! Puedes intentar ejercicios más desafiantes sobre {tema}."
        else:
            return f"Consulta este recurso introductorio sobre {tema}."

    def crear_plan_estudio(self, tema, nivel):
        return f"Plan para {tema} ({nivel}):\n1. Revisión de conceptos clave.\n2. Práctica guiada.\n3. Evaluación semanal."


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Modelos base cargados correctamente.


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
